In [1]:
import findspark
findspark.init('/home/harkirat/spark-3.3.1-bin-hadoop3')

import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lreg').getOrCreate()

22/12/21 17:15:15 WARN Utils: Your hostname, harkirat-QEMU-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 192.168.64.16 instead (on interface enp0s6)
22/12/21 17:15:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 17:15:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
train=spark.read.format('libsvm').load('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

22/12/21 17:15:17 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [4]:
train.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
lr=LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [6]:
lrModel=lr.fit(train)

22/12/21 17:15:21 WARN Instrumentation: [5c011258] regParam is zero, which might cause numerical instability and overfitting.


In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [9]:
train_summary=lrModel.summary

In [10]:
train_summary.r2

0.027839179518600154

In [11]:
train_summary.rootMeanSquaredError

10.16309157133015

# Train/Test Split

In [12]:
all_data=spark.read.format('libsvm').load('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

22/12/21 17:15:21 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [13]:
train_data, test_data=all_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                346|
|   mean| 0.7272198826058304|
| stddev| 10.418624947448272|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [15]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                155|
|   mean|-0.7930114803995245|
| stddev| 10.043088652931582|
|    min|-28.571478869743427|
|    max| 20.874246167942125|
+-------+-------------------+



In [16]:
correct_model=lr.fit(train_data)

22/12/21 17:15:23 WARN Instrumentation: [219dc3bf] regParam is zero, which might cause numerical instability and overfitting.


In [17]:
test_results=correct_model.evaluate(test_data)

In [18]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -28.55363683451943|
|-21.026378279382566|
|-22.041779514991646|
|-19.546757958962733|
| -21.48951879006535|
|-19.833187782434745|
|-16.564985933173585|
|-15.075956019996955|
|-20.618206238148822|
|-14.073228666753781|
|-18.887397444971977|
|    -17.92180275873|
|  -18.0614357010572|
|  -13.5260538989403|
|-13.033980872919095|
|-14.024429252181648|
|-15.737311800934622|
| -10.74013781113162|
|-13.389749071722969|
|-10.821741489126337|
+-------------------+
only showing top 20 rows



In [19]:
test_results.rootMeanSquaredError

10.158928160109339

In [20]:
unlabeled_data=test_data.select('features')

In [21]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [22]:
predictions=correct_model.transform(unlabeled_data)

In [23]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.01784203522399508|
|(10,[0,1,2,3,4,5,...|   1.141817505109141|
|(10,[0,1,2,3,4,5,...|  2.2590167253771094|
|(10,[0,1,2,3,4,5,...|  0.3784653359989695|
|(10,[0,1,2,3,4,5,...|   2.643596317166769|
|(10,[0,1,2,3,4,5,...|  1.5579742164301138|
|(10,[0,1,2,3,4,5,...| -1.2386402554909322|
|(10,[0,1,2,3,4,5,...|  -1.950536244212592|
|(10,[0,1,2,3,4,5,...|   4.356775962696091|
|(10,[0,1,2,3,4,5,...|  -2.012430374267709|
|(10,[0,1,2,3,4,5,...|  3.1067124123486773|
|(10,[0,1,2,3,4,5,...|  2.1897144864907547|
|(10,[0,1,2,3,4,5,...|   2.337920088008631|
|(10,[0,1,2,3,4,5,...| -1.9113308944909186|
|(10,[0,1,2,3,4,5,...| -0.8331070222396725|
|(10,[0,1,2,3,4,5,...|  0.9845011880770336|
|(10,[0,1,2,3,4,5,...|   2.759463075542518|
|(10,[0,1,2,3,4,5,...|  -2.033089188119577|
|(10,[0,1,2,3,4,5,...|  0.8311732828667797|
|(10,[0,1,2,3,4,5,...| -1.645914

# Example 2

In [24]:
spark=SparkSession.builder.appName('lr_ex_2').getOrCreate()

22/12/21 17:15:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [25]:
data=spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [26]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [27]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [28]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [29]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [30]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [31]:
assembler=VectorAssembler(inputCols=['Avg Session Length',
                                     'Time on App',
                                     'Time on Website',
                                     'Length of Membership'],
                          outputCol='features')

In [32]:
output=assembler.transform(data)

In [33]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [34]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [35]:
final_data=output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [36]:
train, test = final_data.randomSplit([0.7,0.3])

In [37]:
train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean| 500.08731783802625|
| stddev|  79.90010304218212|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+



In [38]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean| 497.42083652962816|
| stddev|  78.10467937640134|
|    min| 256.67058229005585|
|    max|  689.7876041747194|
+-------+-------------------+



In [39]:
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [40]:
lr_model=lr.fit(train)

22/12/21 17:15:26 WARN Instrumentation: [21fbf40f] regParam is zero, which might cause numerical instability and overfitting.


In [41]:
test_results=lr_model.evaluate(test)

In [42]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.318088462231344|
| -5.435251544959158|
|-17.268962275730587|
|-21.702849845088963|
| -7.441910583194613|
| 18.625437009892778|
|0.20882231084647174|
| -14.48298079548772|
|   6.45771869253457|
| -7.293466301576586|
| -2.154984583198427|
| 1.9481831513865586|
| -4.070653501160905|
|-2.7113591587956307|
| -17.48390863122637|
| -5.833406920537641|
|  7.806416421383005|
| -0.245923497818751|
| 16.881995608044235|
|  2.550594028336377|
+-------------------+
only showing top 20 rows



In [43]:
test_results.rootMeanSquaredError

9.789582817769213

In [44]:
test_results.r2

0.9841809728355945

# Example 3

In [45]:
spark=SparkSession.builder.appName('lr_ex_3').getOrCreate()
data=spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv', inferSchema=True, header=True)

22/12/21 17:16:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [47]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [48]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [49]:
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [50]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [51]:
from pyspark.ml.feature import StringIndexer

In [53]:
indexer=StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed=indexer.fit(data).transform(data)

In [54]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [56]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [57]:
assembler=VectorAssembler(inputCols=['Age',
                                     'Tonnage',
                                     'passengers',
                                     'length',
                                     'cabins',
                                     'passenger_density',
                                     'cruise_cat'],
                          outputCol='features')

In [59]:
output=assembler.transform(indexed)

In [61]:
output.select('features', 'crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [62]:
final_data=output.select('features', 'crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [73]:
train, test = final_data.randomSplit([0.8,0.2])

In [74]:
train.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              130|
|   mean|7.822153846153857|
| stddev|3.466046825463113|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [75]:
test.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                28|
|   mean| 7.664285714285714|
| stddev|3.7355672171522274|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [76]:
lr=LinearRegression(labelCol='crew')

In [77]:
trained_ship_model=lr.fit(train)

22/12/21 17:33:17 WARN Instrumentation: [bc1a4b64] regParam is zero, which might cause numerical instability and overfitting.


In [78]:
ship_results=trained_ship_model.evaluate(test)

In [79]:
ship_results.rootMeanSquaredError

0.6815319553871341

In [80]:
ship_results.r2

0.9654813657993128

In [81]:
from pyspark.sql.functions import corr

In [82]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

